In [ ]:
# This cell is mandatory in all Dymos documentation notebooks.
missing_packages = []
try:
    import openmdao.api as om
except ImportError:
    if 'google.colab' in str(get_ipython()):
        !python -m pip install openmdao[notebooks]
    else:
        missing_packages.append('openmdao')
try:
    import dymos as dm
except ImportError:
    if 'google.colab' in str(get_ipython()):
        !python -m pip install dymos
    else:
        missing_packages.append('dymos')
try:
    import pyoptsparse
except ImportError:
    if 'google.colab' in str(get_ipython()):
        !pip install -q condacolab
        import condacolab
        condacolab.install_miniconda()
        !conda install -c conda-forge pyoptsparse
    else:
        missing_packages.append('pyoptsparse')
if missing_packages:
    raise EnvironmentError('This notebook requires the following packages '
                           'please install them and restart this notebook\'s runtime: {",".join(missing_packages)}')

(examples:cart_pole)=
# Cart-Pole Optimal Control

This example is authored by Shugo Kaneko and Bernardo Pacini of [MDO Lab](https://mdolab.engin.umich.edu/).
The cart-pole problem is a classic control problem that has been used in many literature and textbook.  TODO: add ref.
We consider a cart with a pole attached, which can rotate freely around the cart.
The external force (control input) is applied to the cart in $x$-direction.

Our goal is to bring the cart-pole system from an initial state to a terminal state with minimum control efforts.
Here, the initial state is the stable stationally point (the cart at a stop with the pole vertically down), and the terminal state is the unstable stationally state (the cart at a stop but the pole is vertically up).
Frictions are ignored to simplify the problem.

![Cart-pole optimal control from the initial state to the terminal state.](cartpole-ic-tc.png)

## Trajectory Optimization Problem

We use the quadratic objective function to approximately minimize the total control effort:
\begin{equation}
    J = \int_{t_0}^{t_f} F(t)^2 dt ~~ \rightarrow ~ \min
\end{equation}
where $F(t)$ is the external force, $t_0$ is the initial time, and $t_f$ is the final time.

### Dynamics

The equation of motion of the cart-pole is given by

\begin{equation}
    \begin{bmatrix}
        \ddot{x} \\ \ddot{\theta}
    \end{bmatrix} =
    \begin{bmatrix}
        \cos \theta & \ell  \\ m_1 + m_2 & m_2 \ell \cos \theta
    \end{bmatrix}^{-1}
    \begin{bmatrix}
        -g \sin \theta \\ F + m_2 \ell \dot{\theta}^2 \sin \theta
    \end{bmatrix}
\end{equation}

where $x$ is the cart location and $\theta$ is the pole angle, $m_1$ is the cart mass, $m_2$ is the pole mass, and $\ell$ is the pole length.
![Schematic of the cart-pole system.](cartpole-dynamics.png)

Now, we need to convert the E.o.M, which is a second-order ODE, to a first-order ODE.
To do so, we define our state vector to be $X = [x, \dot{x}, \theta, \dot{\theta}]^T$.
We also add an "energy" state $e$ to keep track of the accumulated control input by setting $\dot{e} = F^2$.
With this additional state, the objective function is transformed to the final state value as

\begin{equation}
    J = \int_{t_0}^{t_f} \dot{e} ~dt = e_f
\end{equation}

by setting $e_0 = 0$.
To summarize, the ODE for the cart-pole system is given by

\begin{equation}
    \begin{bmatrix}
        \dot{x} \\ \dot{\theta} \\ \ddot{x} \\ \ddot{\theta} \\ \dot{e} 
    \end{bmatrix} =
    f \left(
        \begin{bmatrix}
        x \\ \theta \\ \dot{x} \\ \dot{\theta} \\ e 
    \end{bmatrix}
    \right)=
    \begin{bmatrix}
        \dot{x}  \\
        \dot{\theta} \\
        \frac{-m_2 g \sin \theta \cos \theta - (F + m_2 \ell \dot{\theta}^2 \sin \theta)}{m_2 \cos^2 \theta - (m_1 + m_2)} \\
        \frac{(m_1 + m_2) g \sin \theta + \cos \theta (F + m_1 \ell \dot{\theta}^2 \sin \theta)}{m_2 \ell \cos^2 \theta - (m_1 + m_2) \ell} \\ 
        F^2 \\ 
    \end{bmatrix}
\end{equation}

### Initial and terminal conditions
Initial state variables are all zero at $t_0 = 0$.
The terminal conditions at time $t_f$ are
\begin{align}
    x_f &= d \\
    \dot{x}_f &= 0 \\
    \theta_f &= \pi \\
    \dot{\theta_f} &= 0
\end{align}

### Parameters
The fixed parameters are summarized as follows.

| Parameter      | Value       | Units        | Description                                |
|----------------|-------------|--------------|--------------------------------------------|
| $m_1$          | 1.0         | $kg$         | Cart mass                                  |
| $m_2$          | 0.3         | $kg$         | Pole mass                                  |
| $\ell$         | 0.5         | $m$          | Pole length                                |
| $d$            | 2           | $m$          | Cart target location                       |
| $t_f$          | 2           | $s$          | Final time                                 |

## Implementing the ODE
We first implement the cart-pole ODE as an `ExplicitComponent`


In [ ]:
import openmdao.api as om
from dymos.models.eom import FlightPathEOM2D
from dymos.examples.min_time_climb.prop import PropGroup
from dymos.models.atmosphere import USatm1976Comp
from dymos.examples.min_time_climb.doc.aero_partial_coloring import AeroGroup

class MinTimeClimbODE(om.Group):

    def initialize(self):
        self.options.declare('num_nodes', types=int)
        self.options.declare('fd', types=bool, default=False, desc='If True, use fd for partials')
        self.options.declare('partial_coloring', types=bool, default=False,
                             desc='If True and fd is True, color the approximated partials')

    def setup(self):
        nn = self.options['num_nodes']

        self.add_subsystem(name='atmos',
                           subsys=USatm1976Comp(num_nodes=nn, h_def='geodetic'),
                           promotes_inputs=['h'])

        self.add_subsystem(name='aero',
                           subsys=AeroGroup(num_nodes=nn,
                                            fd=self.options['fd'],
                                            partial_coloring=self.options['partial_coloring']),
                           promotes_inputs=['v', 'alpha', 'S'])

        self.connect('atmos.sos', 'aero.sos')
        self.connect('atmos.rho', 'aero.rho')

        self.add_subsystem(name='prop',
                           subsys=PropGroup(num_nodes=nn),
                           promotes_inputs=['h', 'Isp', 'throttle'])

        self.connect('aero.mach', 'prop.mach')

        self.add_subsystem(name='flight_dynamics',
                           subsys=FlightPathEOM2D(num_nodes=nn),
                           promotes_inputs=['m', 'v', 'gam', 'alpha'])

        self.connect('aero.f_drag', 'flight_dynamics.D')
        self.connect('aero.f_lift', 'flight_dynamics.L')
        self.connect('prop.thrust', 'flight_dynamics.T')


## Building and running the problem

In the following code we follow the following process to solve the
problem:

In [ ]:
import matplotlib.pyplot as plt

import openmdao.api as om

import dymos as dm
from dymos.examples.plotting import plot_results

#
# Instantiate the problem and configure the optimization driver
#
p = om.Problem(model=om.Group())

p.driver = om.pyOptSparseDriver()
p.driver.options['optimizer'] = 'SLSQP'
p.driver.declare_coloring()

#
# Instantiate the trajectory and phase
#
traj = dm.Trajectory()

phase = dm.Phase(ode_class=MinTimeClimbODE,
                 transcription=dm.GaussLobatto(num_segments=15, compressed=False))

traj.add_phase('phase0', phase)

p.model.add_subsystem('traj', traj)

#
# Set the options on the optimization variables
# Note the use of explicit state units here since much of the ODE uses imperial units
# and we prefer to solve this problem using metric units.
#
phase.set_time_options(fix_initial=True, duration_bounds=(50, 400),
                       duration_ref=100.0)

phase.add_state('r', fix_initial=True, lower=0, upper=1.0E6, units='m',
                ref=1.0E3, defect_ref=1.0E3,
                rate_source='flight_dynamics.r_dot')

phase.add_state('h', fix_initial=True, lower=0, upper=20000.0, units='m',
                ref=1.0E2, defect_ref=1.0E2,
                rate_source='flight_dynamics.h_dot')

phase.add_state('v', fix_initial=True, lower=10.0, units='m/s',
                ref=1.0E2, defect_ref=1.0E2,
                rate_source='flight_dynamics.v_dot')

phase.add_state('gam', fix_initial=True, lower=-1.5, upper=1.5, units='rad',
                ref=1.0, defect_ref=1.0,
                rate_source='flight_dynamics.gam_dot')

phase.add_state('m', fix_initial=True, lower=10.0, upper=1.0E5, units='kg',
                ref=1.0E3, defect_ref=1.0E3,
                rate_source='prop.m_dot')

phase.add_control('alpha', units='deg', lower=-8.0, upper=8.0, scaler=1.0,
                  rate_continuity=True, rate_continuity_scaler=100.0,
                  rate2_continuity=False)

phase.add_parameter('S', val=49.2386, units='m**2', opt=False, targets=['S'])
phase.add_parameter('Isp', val=1600.0, units='s', opt=False, targets=['Isp'])
phase.add_parameter('throttle', val=1.0, opt=False, targets=['throttle'])

#
# Setup the boundary and path constraints
#
phase.add_boundary_constraint('h', loc='final', equals=20000, scaler=1.0E-3)
phase.add_boundary_constraint('aero.mach', loc='final', equals=1.0)
phase.add_boundary_constraint('gam', loc='final', equals=0.0)

phase.add_path_constraint(name='h', lower=100.0, upper=20000, ref=20000)
phase.add_path_constraint(name='aero.mach', lower=0.1, upper=1.8)

# Minimize time at the end of the phase
phase.add_objective('time', loc='final', ref=1.0)

p.model.linear_solver = om.DirectSolver()

#
# Setup the problem and set the initial guess
#
p.setup(check=True)

p['traj.phase0.t_initial'] = 0.0
p['traj.phase0.t_duration'] = 500

p.set_val('traj.phase0.states:r', phase.interp('r', [0.0, 50000.0]))
p.set_val('traj.phase0.states:h', phase.interp('h', [100.0, 20000.0]))
p.set_val('traj.phase0.states:v', phase.interp('v', [135.964, 283.159]))
p.set_val('traj.phase0.states:gam', phase.interp('gam', [0.0, 0.0]))
p.set_val('traj.phase0.states:m', phase.interp('m', [19030.468, 10000.]))
p.set_val('traj.phase0.controls:alpha', phase.interp('alpha', [0.0, 0.0]))

#
# Solve for the optimal trajectory
#
dm.run_problem(p, simulate=True)



In [ ]:
sol = om.CaseReader('dymos_solution.db').get_case('final')
sim = om.CaseReader('dymos_simulation.db').get_case('final')

plot_results([('traj.phase0.timeseries.time', 'traj.phase0.timeseries.states:h',
               'time (s)', 'altitude (m)'),
              ('traj.phase0.timeseries.time', 'traj.phase0.timeseries.controls:alpha',
               'time (s)', 'alpha (deg)')],
             title='Supersonic Minimum Time-to-Climb Solution',
             p_sol=sol, p_sim=sim)

plt.show()

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('traj.phase0.t_duration'), 321.0, tolerance=1.0E-1)

## References

```{bibliography}
:filter: docname in docnames
```